In [ ]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [ ]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 85% 73.0M/85.7M [00:00<00:00, 48.1MB/s]
100% 85.7M/85.7M [00:01<00:00, 84.2MB/s]


In [ ]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [ ]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 7.8 MB/s 
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 6.4 MB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 39.5 MB/s 
     |████████████████████████████████| 840 kB 8.7 MB/s 
     |████████████████████████████████| 120 kB 69.2 MB/s 
     |████████████████████████████████| 234 kB 45.6 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
     |████████████████████████████████| 3.4 MB 40.3 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 352 kB 59.9 MB/s 
     |████████████████████████████████| 462 kB 49.5 MB/s 
     |████████████████

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [ ]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [ ]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [ ]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [ ]:
spec = model_spec.get('efficientdet_lite3')

In [27]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/50
397/397 [==============================] - 240s 499ms/step - det_loss: 0.9414 - cls_loss: 0.7425 - box_loss: 0.0040 - reg_l2_loss: 0.0938 - loss: 1.0351 - learning_rate: 0.0078 - gradient_norm: 3.5079 - val_det_loss: 0.6002 - val_cls_loss: 0.5131 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.0941 - val_loss: 0.6942
Epoch 2/50
397/397 [==============================] - 196s 494ms/step - det_loss: 0.6077 - cls_loss: 0.5054 - box_loss: 0.0020 - reg_l2_loss: 0.0943 - loss: 0.7019 - learning_rate: 0.0075 - gradient_norm: 3.2182 - val_det_loss: 0.4804 - val_cls_loss: 0.4436 - val_box_loss: 7.3611e-04 - val_reg_l2_loss: 0.0944 - val_loss: 0.5748
Epoch 3/50
397/397 [==============================] - 195s 491ms/step - det_loss: 0.5383 - cls_loss: 0.4521 - box_loss: 0.0017 - reg_l2_loss: 0.0946 - loss: 0.6328 - learning_rate: 0.0075 - gradient_norm: 3.2185 - val_det_loss: 0.4479 - val_cls_loss: 0.3873 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0947 - val_loss: 0.5426
Epoch 4/50
397/397

In [28]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 10034.77288532257


In [29]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 15s 100ms/step

{'AP': 0.8398091, 'AP50': 0.9415507, 'AP75': 0.8927373, 'APs': -1.0, 'APm': 0.6279617, 'APl': 0.85373497, 'ARmax1': 0.75979257, 'ARmax10': 0.88312644, 'ARmax100': 0.8897697, 'ARs': -1.0, 'ARm': 0.67333335, 'ARl': 0.9020386, 'AP_/10': 0.788982, 'AP_/20': 0.83456755, 'AP_/50': 0.8669382, 'AP_/100': 0.93162507, 'AP_/200': 0.8833593, 'AP_/500': 0.8195699, 'AP_/2000': 0.7536219}


In [30]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 17.591188192367554


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [ ]:
model.export(export_dir='/content/')